In [1]:
import pandas as pd
import csv
import json

In [2]:
Sea_level=pd.read_csv('D:/Ruchita/NCI/sem1/DAP/DATASET1/Sea_level.csv')
Sea_level.head()

,Date,Global_mean_SL
0,1880-01-15,-183.0
1,1880-02-15,-171.1
2,1880-03-15,-164.3
3,1880-04-15,-158.2
4,1880-05-15,-158.7


In [4]:
with open("D:/Ruchita/NCI/sem1/DAP/DATASET1/Sea_level.csv","r") as f:
    reader= csv.reader(f)
    next(reader)
    Sea_level=[]
    for row in reader:
        Sea_level.append({"Date":row[0],"Global_mean_SL":row[1]})

In [5]:
with open("Finalfile.json","w") as f:
    json.dump(Sea_level, f, indent=4)


In [6]:
Sea_level

[{'Date': '1880-01-15', 'Global_mean_SL': '-183.0'},
 {'Date': '1880-02-15', 'Global_mean_SL': '-171.1'},
 {'Date': '1880-03-15', 'Global_mean_SL': '-164.3'},
 {'Date': '1880-04-15', 'Global_mean_SL': '-158.2'},
 {'Date': '1880-05-15', 'Global_mean_SL': '-158.7'},
 {'Date': '1880-06-15', 'Global_mean_SL': '-159.6'},
 {'Date': '1880-07-15', 'Global_mean_SL': '-159.6'},
 {'Date': '1880-08-15', 'Global_mean_SL': '-161.8'},
 {'Date': '1880-09-15', 'Global_mean_SL': '-158.9'},
 {'Date': '1880-10-15', 'Global_mean_SL': '-150.3'},
 {'Date': '1880-11-15', 'Global_mean_SL': '-144.5'},
 {'Date': '1880-12-15', 'Global_mean_SL': '-134.7'},
 {'Date': '1881-01-15', 'Global_mean_SL': '-136.4'},
 {'Date': '1881-02-15', 'Global_mean_SL': '-139.2'},
 {'Date': '1881-03-15', 'Global_mean_SL': '-145.2'},
 {'Date': '1881-04-15', 'Global_mean_SL': '-151.1'},
 {'Date': '1881-05-15', 'Global_mean_SL': '-152.2'},
 {'Date': '1881-06-15', 'Global_mean_SL': '-154.2'},
 {'Date': '1881-07-15', 'Global_mean_SL': '-14

In [7]:
#To connect mongodb 
from pymongo import MongoClient
import pymongo as pym

client = MongoClient('127.0.0.1', 27017)
db = client['Sea_level']
Sea_l = db['Sea_level_C']
result = Sea_l.insert_many(Sea_level)


In [8]:
Sea_cursor = Sea_l.find()
Sea_datalist = list(Sea_cursor)
Sea_df = pd.DataFrame(Sea_datalist)

In [9]:
Sea_df.head()

,_id,Date,Global_mean_SL
0,5ff7c32e0b9404e1c5b69519,1880-01-15,-183.0
1,5ff7c32e0b9404e1c5b6951a,1880-02-15,-171.1
2,5ff7c32e0b9404e1c5b6951b,1880-03-15,-164.3
3,5ff7c32e0b9404e1c5b6951c,1880-04-15,-158.2
4,5ff7c32e0b9404e1c5b6951d,1880-05-15,-158.7


In [10]:
Sea_df1=Sea_df.drop(['_id'],axis=1) 
Sea_df1


,Date,Global_mean_SL
0,1880-01-15,-183.0
1,1880-02-15,-171.1
2,1880-03-15,-164.3
3,1880-04-15,-158.2
4,1880-05-15,-158.7
...,...,...
1603,2013-08-15,71.6
1604,2013-09-15,68.8
1605,2013-10-15,66.4
1606,2013-11-15,59.7


In [12]:
Sea_df1.to_csv('M_Sea_level.csv', index = None)

In [13]:
import psycopg2
try:
    dbConnection = psycopg2.connect(
        user = "dap",
        password = "dap",
        host = "192.168.56.30",
        port = "5432",
        database = "postgres")
    dbConnection.set_isolation_level(0) # AUTOCOMMIT
    dbCursor = dbConnection.cursor()
    dbCursor.execute('CREATE DATABASE population;')
    dbCursor.close()
except (Exception , psycopg2.Error) as dbError :
    print ("Error while connecting to PostgreSQL", dbError)
finally:
    if(dbConnection): dbConnection.close()

In [14]:
createString = """
CREATE TABLE Sea_level(
    Date varchar(255),
    Global_mean_SL varchar(255)
);
"""
try: 
    dbConnection = psycopg2.connect(
        user = "dap",
        password = "dap",
        host = "192.168.56.30",
        port = "5432",
        database = "population")
    dbConnection.set_isolation_level(0) # AUTOCOMMIT
    dbCursor = dbConnection.cursor()
    dbCursor.execute(createString)
    dbCursor.close()
except (Exception , psycopg2.Error) as dbError :
    print ("Error while connecting to PostgreSQL", dbError)
finally:
    if(dbConnection): dbConnection.close()


In [18]:
import csv
try:
    dbConnection = psycopg2.connect(
        user = "dap",
        password = "dap",
        host = "192.168.56.30",
        port = "5432",
        database = "population")
    dbConnection.set_isolation_level(0) # AUTOCOMMIT
    dbCursor = dbConnection.cursor()
    insertString = "INSERT INTO Sea_level VALUES ('{}',"+"'{}')"
    with open('D:/Ruchita/NCI/sem1/DAP/DATASET1/M_Sea_level.csv', 'r') as f:
        reader = csv.reader(f)
        next(reader) # skip the header
        for row in reader:
            dbCursor.execute(insertString.format(*row))
    dbConnection.commit()
    dbCursor.close()
except (Exception , psycopg2.Error) as dbError :
    print ("Error:", dbError)
finally:
    if(dbConnection): dbConnection.close()

In [19]:
import pandas.io.sql as sqlio
query = """ SELECT *  FROM Sea_level;"""
try:
    dbConnection = psycopg2.connect(
        user = "dap",
        password = "dap",
        host = "192.168.56.30",
        port = "5432",
        database = "population")
    Sea_level = sqlio.read_sql_query(query, dbConnection) 
except (Exception , psycopg2.Error) as dbError :
    print ("Error:", dbError)
finally:
    if(dbConnection): dbConnection.close()

In [20]:
Sea_level

,date,global_mean_sl
0,1880-01-15,-183.0
1,1880-02-15,-171.1
2,1880-03-15,-164.3
3,1880-04-15,-158.2
4,1880-05-15,-158.7
...,...,...
1603,2013-08-15,71.6
1604,2013-09-15,68.8
1605,2013-10-15,66.4
1606,2013-11-15,59.7
